# Pull battrovic data

The purpose of this script is to pull data from the following [hyperlinked website](https://barttorvik.com/trank.php#).
This awesome source contains data on all D1 March Madness teams and is customizable to:
* team
* conference
* state
* by conf
* by coach
* start (date)
* end (date)
* vs. Top {} or {operator} {dropdown quadrant}
* Venue
* Type
* Min. Games

Anyways, they're awesome, and we'll be scraping their data.

------

<p>Author: PJ Gibson</p>
<p>Create Date: 2023-03-14</p>

## 1. Import Libraries

In [166]:
# Best lib for numeric manipulation in my opinion
import numpy as np

# Gotta import pandas
import pandas as pd

# For API pulls
import requests

# Import sklearn preprocessing just for a nice normalizing function
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

# For sleeping in between requests
import time

# For visualization of how fast things are moving
from tqdm import tqdm

pd.set_option('display.max_columns', None)

## 2. Define Manual Specs

In [174]:
 # Had to look each of these up manually.  I chose the day before the tournament started as the marker for end_date
ncaa_pre_madness_dates = pd.DataFrame([(2008, '20071101', '20080317' ),
                                       (2009, '20081101', '20090316' ),
                                       (2010, '20091101', '20100315' ),
                                       (2011, '20101101', '20110314' ),
                                       (2012, '20111101', '20120312' ),
                                       (2013, '20121101', '20130317' ),
                                       (2014, '20131101', '20140317' ),
                                       (2015, '20141101', '20150316' ),
                                       (2016, '20151101', '20160314' ),
                                       (2017, '20161101', '20170313' ),
                                       (2018, '20171101', '20180312' ),
                                       (2019, '20181101', '20190318' ),
                                       (2021, '20201101', '20210317' ),
                                       (2022, '20211101', '20220314' ),
                                       (2023, '20221101', '20230313')], columns = ['year','start_date','end_date'])

## 3. Define function to guide data wrangling

In [192]:
def process_response( response, iter_year, iter_type):
    """Processes the API response to the barttorvik website into clean dataframe

    Args:
        response: the response to an API request for the website:
            https://barttorvik.com/trank.php#
        iter_year: the year in which the march madness tournament took place
            note the regular season started winter of the year prior
        iter_type: either "P" or "R" indicating post-season or regular season respectively


    Returns:
        A pandas dataframe with cleaned output.

    """

    # Convert response into pandas dataframe
    df = pd.read_html(response.text)[0]

    # Dig into the column indices and cleanup
    multi_index_columns = pd.DataFrame(list(df.columns), columns=['upper','lower'])
    multi_index_columns['upper'] = multi_index_columns.copy()['upper'].str.extract('(\d+\.\d+)$')
    multi_index_columns['lower'] = multi_index_columns.copy()['lower'].str.extract('(?:.*?\>)?(.*)$')

    # Redefine columns
    df.columns = list(multi_index_columns['lower'])

    # Clean up data
    df = df.query('Rk != "Rk"')

    # Clean up very confusing format for cell
    df['Rec'] = df.copy()['Rec'].str.extract('(?:.*?\>)?(.*)$')

    # Wrangle the record into winning percentage
    df[['Overall_Record','Conf_Record']] = df.copy()['Rec'].str.split(' ', expand=True)
    df = df.drop('Rec',axis=1)

    if iter_type == 'R':
        df['conference_win_pct'] = df.copy()['Conf_Record'].str.split('\–',expand=True)[0].astype(int) / df.copy()['Conf_Record'].str.split('\–',expand=True).astype(int).sum(axis=1)

    else:
        df['conference_win_pct'] = np.nan

    df['overall_win_pct'] = df.copy()['Overall_Record'].str.split('\–',expand=True)[0].astype(int) / df.copy()['G'].astype(int)

    # Many columns contain 2 values: a floating decimal score, and a rank amongst other D1 teams
    #### we will only be using the floating decimal score
    rank_cols = ['AdjOE', 'AdjDE', 'Barthag', 'EFG%',
               'EFGD%', 'TOR', 'TORD', 'ORB', 'DRB', 'FTR', 'FTRD', '2P%', '2P%D',
               '3P%', '3P%D', 'Adj T.', 'WAB']

    # For each column that contains the 2 values...
    for col in rank_cols:

        # Split up the floating decimal and the rank, only take the floating decimal
        df[col] = df.copy()[col].str.split(' ', expand=True)[0]

        # Convert to float (previously string type)
        df[col] = df.copy()[col].astype(float)


    # Query things down to only teams participating in march madness
    # df = df[df.copy()['Team'].str.contains('\d')]  

    # # Normalize all together
    # x = df[rank_cols]
    # x_scaled = min_max_scaler.fit_transform(x)
    # df[rank_cols] = x_scaled

    df['exit'] = df.copy()['Team'].str.split(',',expand=True)[1]
    df['df_quad_rank'] = df.copy()['Team'].str.extract('(\d+)')
    df['Team'] = df['Team'].str.split('(.*?)(?:(?: \d)|$)',expand=True)[1]

    # Drop unused columns
    df = df.drop(['Overall_Record','Conf_Record'],axis=1)

    # Add type
    df['Type'] = iter_type
    df['Year'] = iter_year

    # Output the dataframe
    return df

## 4. Collect Data

In [193]:
# Initialized empty output
list_of_dfs = []

# For each year in our predefined bracket...
for i in tqdm(np.arange(0,len(ncaa_pre_madness_dates))):
    
    # Define current values in the iteration
    iter_year = ncaa_pre_madness_dates.loc[i,'year']
    iter_start = ncaa_pre_madness_dates.loc[i,'start_date']
    iter_end = ncaa_pre_madness_dates.loc[i,'end_date']
    
    # For regular season and playoffs...
    for iter_type in ['R','P']:
        
        # Define the URL we will perform API Pull on
        url = f'https://barttorvik.com/trank.php?year={iter_year}&sort=&hteam=&t2value=&conlimit=All&state=All&begin={iter_start}&end={iter_end}&top=0&revquad=0&quad=5&venue=All&type={iter_type}&mingames=0#'

        # Perform API query
        response = requests.get(url)
        
        # Apply our custom processing function on the data
        output_df = process_response( response, iter_year, iter_type)
    
        # Append to our output list
        list_of_dfs.append(output_df)
        
        # Sleep (common courtesy for repeated API pulls as not to overwhelm systems)
        time.sleep(2)

  0%|                                                                                           | 0/15 [00:00<?, ?it/s]<ipython-input-192-243effb9270a>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rec'] = df.copy()['Rec'].str.extract('(?:.*?\>)?(.*)$')
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Overall_Record','Conf_Record']] = df.copy()['Rec'].str.split(' ', expand=True)
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to 

<ipython-input-192-243effb9270a>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rec'] = df.copy()['Rec'].str.extract('(?:.*?\>)?(.*)$')
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Overall_Record','Conf_Record']] = df.copy()['Rec'].str.split(' ', expand=True)
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

 47%|██████████████████████████████████████▋                                            | 7/15 [00:56<01:06,  8.28s/it]<ipython-input-192-243effb9270a>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rec'] = df.copy()['Rec'].str.extract('(?:.*?\>)?(.*)$')
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Overall_Record','Conf_Record']] = df.copy()['Rec'].str.split(' ', expand=True)
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to 

<ipython-input-192-243effb9270a>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rec'] = df.copy()['Rec'].str.extract('(?:.*?\>)?(.*)$')
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Overall_Record','Conf_Record']] = df.copy()['Rec'].str.split(' ', expand=True)
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [01:58<00:08,  8.74s/it]<ipython-input-192-243effb9270a>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Rec'] = df.copy()['Rec'].str.extract('(?:.*?\>)?(.*)$')
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Overall_Record','Conf_Record']] = df.copy()['Rec'].str.split(' ', expand=True)
<ipython-input-192-243effb9270a>:35: SettingWithCopyWarning: 
A value is trying to 

## 5. Concat data

In [196]:
# Concat our results into one long dataframe
out = pd.concat(list_of_dfs)

## 6. Save!

In [197]:
# Save our data
out.to_csv('../Data/01_SOURCE/barttorvik_data.csv',header=True,index=False)